In [1]:
import os
from PIL import Image
import numpy as np
import tensorrt as trt
import gc
import torch
import matplotlib.pyplot as plt

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
Image_size = 256
storage_dir = "./storage"
file_list = os.listdir(storage_dir+"/cache_"+str(Image_size))
print("Number of Images",len(file_list))
pics = [ storage_dir + "/cache_"+str(Image_size) + "/" +str(i) for i in file_list]

Number of Images 1029


In [4]:
np_pics = np.asarray(Image.open(pics[0]))
brain_tiles = np_pics.reshape(1,4,Image_size,Image_size)
for i in range(1028):
    img = np.asarray(Image.open(pics[i+1]))
    if (img.shape == (256, 256, 3)):
        brain_tiles = np.concatenate((brain_tiles,img.reshape(1,4,Image_size,Image_size)),axis=0)
    

In [5]:
brain_tiles = torch.from_numpy(brain_tiles).float()

In [6]:
brain_tiles = brain_tiles[:1024,:3,:,:]

In [7]:
brain_tiles.shape

torch.Size([1, 3, 256, 256])

In [9]:
num_gpu = 2
chunk = brain_tiles.shape[0]//num_gpu

In [10]:
brain_batches = [brain_tiles[i:i+chunk,:,:,:] for i in range(0,brain_tiles.shape[0],chunk)]

In [11]:
len(brain_batches)

2

In [12]:
brain_batches[0].shape

torch.Size([512, 3, 256, 256])

In [13]:
brain_batches[1].shape

torch.Size([512, 3, 256, 256])

In [14]:
from pycuda import driver

driver.init()
ngpus = driver.Device.count()

In [15]:
# ngpus

In [16]:
ctx=[driver.Device(gpuid).make_context() for gpuid in range(ngpus)]

In [17]:
ctx

In [17]:
# devices = [elt.get_device() for elt in ctx]

In [18]:
# devices

In [19]:
# devices[1].pci_bus_id()

In [20]:
# import pycuda.autoinit

In [21]:
# pycuda.autoinit.device.pci_bus_id()

In [18]:
from concurrent.futures import ThreadPoolExecutor
import TensorRT.inference as inf
print(os.cpu_count())

6


In [19]:
import pycuda.driver as cuda

import nvidia_smi
nvidia_smi.nvmlInit()
from ctypes import cdll, c_char_p
libcudart = cdll.LoadLibrary('libcudart.so')
libcudart.cudaGetErrorString.restype = c_char_p
def cudaSetDevice(device_idx):
    ret = libcudart.cudaSetDevice(device_idx)
    if ret != 0:
        error_string = libcudart.cudaGetErrorString(ret)
        raise RuntimeError("cudaSetDevice: " + error_string)


In [34]:
def run_inf(context,h_input, d_input, h_output, d_output, stream,image,gpu_id):
    ctx[gpu_id].push()
    inf.load_images_to_buffer(image, h_input)
    cuda.memcpy_htod_async(d_input, h_input, stream)
    context.execute_v2(bindings=[int(d_input), int(d_output)])
    cuda.memcpy_dtoh_async(h_output, d_output, stream)
    
def run_batch_inf(engine,images,gpu_id,batch_sizes=4):
    
    h_input, d_input, h_output, d_output, stream = inf.allocate_buffers(engine,1,trt.float32,ctx[gpu_id])
    context = engine.create_execution_context()
    input_shape = (batch_sizes,3,256,256)
    context.set_binding_shape(0, input_shape)
    print(images.shape)
    for i in range(0,images.shape[0],batch_sizes):
        run_inf(context,h_input, d_input, h_output, d_output, stream,images[i:i+batch_sizes,:,:,:],gpu_id)
        

In [21]:
TRT_LOGGER = trt.Logger(min_severity =trt.ILogger.INTERNAL_ERROR)
def load_engine(engine_file_path):
    assert os.path.exists(engine_file_path)
    print("Reading engine from file {}".format(engine_file_path))
    with open(engine_file_path, "rb") as f, trt.Runtime(TRT_LOGGER) as runtime:
        return runtime.deserialize_cuda_engine(f.read())


In [22]:
engines = []

In [23]:
engines

[]

In [24]:
len(brain_batches)

2

In [25]:
model_name = "hovernet_256_4_fp16.plan"

In [26]:
cudaSetDevice(0)
engines.append(load_engine(model_name))
cudaSetDevice(1)
engines.append(load_engine(model_name))

Reading engine from file hovernet_256_4_fp16.plan
Reading engine from file hovernet_256_4_fp16.plan


In [ ]:
with ThreadPoolExecutor(2) as executor:
    results = executor.map(run_batch_inf,engines,brain_batches,[0,1])

/tmp/ipykernel_897/375445964.py:13: DeprecationWarning: Use set_input_shape instead.
  context.set_binding_shape(0, input_shape)


In [1]:
for r in results:
    print(r)

NameError: name 'results' is not defined

In [35]:
%%time
for i in range(2):
    run_batch_inf(engines[i%2],brain_batches[i%2],i%2)

/tmp/ipykernel_938/850779005.py:13: DeprecationWarning: Use set_input_shape instead.
  context.set_binding_shape(0, input_shape)


torch.Size([512, 3, 256, 256])
torch.Size([512, 3, 256, 256])
CPU times: user 3.65 s, sys: 62.5 ms, total: 3.71 s
Wall time: 3.71 s


In [30]:
ctx